I. Write a function in your choice of language

Input: 
 - An list  (seqs) of DNA sequences, e.g. [“AAAAGAAAAA”, “CCCAAATTTT”,”GGGGGGGGGGAGA”, “CAAATTT”,…. ]
 - An integer k, e.g. 3

Analysis: Flag a sequence seqs[i] if it shares at least one k-character substring with any other sequence seqs[j] if j != i

Output: A list of integers (i) representing the flagged sequences seq[i]


II. Code change Followup:

Flag a sequence seqs[i] if it shares at least one k-bp sub-sequence with either any other sequence seqs[j] if j=i, or the reverse complement any other sequence seqs[j] if j != i


In [1]:
def extract_kmers(sequence, k):
    '''
    Finds the set of kmers in a sequence
    '''
    subseq = []
    
    for ix,val in enumerate(sequence):
        
        if ix < (len(sequence)-k+1):
            subseq.append(sequence[ix:ix+k])
    
    return set(subseq)
           

def map_index_kmers(seq_list, k):
    '''
    Creates a dictionary of index to sequence (key) and the set of k-mers (value)
    '''
    k_dic = {}

    for idx, seq in enumerate(seq_list):

        k_dic[idx] = extract_kmers(seq, k) # get all kmers in sequence

    return k_dic


def reverse_complement(sequence):
    '''
    Returns the reverse complement of a sequence    
    '''
    comp = ""
    
    # substitute purine and pyrimidine pairs
    for s in sequence:
        if s == 'A':
            comp += 'T'
        elif s == 'T':
            comp += 'A'
        elif s == 'G':
            comp += 'C'
        elif s == 'C':
            comp += 'G'
        else:
            comp += s    # for 'N' or other non-ATCG characters
            
    # reverse the sequence
    comp = comp[::-1]
    
    return comp


def get_reverse_comp(seq_list):
    '''
    Returns a list of the reverse complement of sequences
    '''
    
    reverse_seq_list = []
    
    for x in seq_list:
        reverse_seq_list.append(reverse_complement(x))
       
    return reverse_seq_list


def flag_matching_kmer(seq_list, k):
    '''
    Generates a list of integers represeting the index of sequences that contain at least one 
    k-mer present in another sequence (including the reverse complement).
    '''

    flagged = []
    
    # dictionary mapping kmers to index of sequences
    fwd_kmer_dic = map_index_kmers(seq_list, k)
    
    # dictionary mapping kmers to index of reverse complement of sequences
    reverse_seq_list = get_reverse_comp(seq_list)   
    rev_kmer_dic = map_index_kmers(reverse_seq_list, k)
    
    for idx, seq in enumerate(seq_list):

        for i in range(len(seq_list)):
            if idx != i:                                               # match with non-self sequences
                if fwd_kmer_dic[idx].intersection(fwd_kmer_dic[i]):    # if sequence has matching k-mer in another sequence
                    flagged.append(idx)                                # save index of seqence and move to next sequence
                    break
                elif fwd_kmer_dic[idx].intersection(rev_kmer_dic[i]):  # if sequence has matching k-mer in reverse complement of another sequence
                    flagged.append(idx)                      
                    break

    print_dicts(fwd_kmer_dic, rev_kmer_dic)
    
    return flagged


def print_dicts(seq, rev_comp):
    '''
    Print dictionaries of indexs mapped to k-mer set
    '''
    print("Sequence:")
    for k,v in seq.items():
        print(k, v)

        
    print("\nReverse complement")
    for k,v in rev_comp.items():
        print(k, v)

In [2]:
# Test 1

seq_list = ['AAAAGAAAAA', 'CCCAAATTTT', 'GGGGGGGGGGAGA', 'CAAATTT']
k = 3

print("\nFlagged indexes:", flag_matching_kmer(seq_list, k))

Sequence:
0 {'AGA', 'AAG', 'GAA', 'AAA'}
1 {'TTT', 'ATT', 'AAT', 'CCC', 'CAA', 'CCA', 'AAA'}
2 {'AGA', 'GAG', 'GGG', 'GGA'}
3 {'TTT', 'ATT', 'AAT', 'CAA', 'AAA'}

Reverse complement
0 {'TTT', 'TTC', 'CTT', 'TCT'}
1 {'TTT', 'GGG', 'ATT', 'TTG', 'AAT', 'TGG', 'AAA'}
2 {'TCC', 'CCC', 'TCT', 'CTC'}
3 {'TTT', 'ATT', 'TTG', 'AAT', 'AAA'}

Flagged indexes: [0, 1, 2, 3]


In [3]:
# Test 2

seq_list = ['AAAAGAAAAA', 'CCCAAATTTT', 'GGGGGGGGGGAGA', 'CAAATTT']
k = 4

print("\nFlagged indexes:", flag_matching_kmer(seq_list, k))

Sequence:
0 {'AGAA', 'AAAG', 'AAAA', 'GAAA', 'AAGA'}
1 {'CAAA', 'AATT', 'TTTT', 'CCCA', 'ATTT', 'CCAA', 'AAAT'}
2 {'GGGG', 'GGGA', 'GAGA', 'GGAG'}
3 {'CAAA', 'ATTT', 'AATT', 'AAAT'}

Reverse complement
0 {'TTCT', 'TTTT', 'TTTC', 'CTTT', 'TCTT'}
1 {'TTGG', 'AATT', 'TGGG', 'AAAA', 'ATTT', 'TTTG', 'AAAT'}
2 {'TCTC', 'TCCC', 'CTCC', 'CCCC'}
3 {'TTTG', 'ATTT', 'AATT', 'AAAT'}

Flagged indexes: [0, 1, 3]


In [4]:
# Test 3

seq_list = ['AAAAAAAAAA', 'CCCCCCTTT', 'GGGGGGGGGGGA']
k = 4

print("\nFlagged indexes:", flag_matching_kmer(seq_list, k))

Sequence:
0 {'AAAA'}
1 {'CCCT', 'CTTT', 'CCCC', 'CCTT'}
2 {'GGGG', 'GGGA'}

Reverse complement
0 {'TTTT'}
1 {'GGGG', 'AAGG', 'AGGG', 'AAAG'}
2 {'TCCC', 'CCCC'}

Flagged indexes: [1, 2]


In [5]:
# Test 4

seq_list = ['ATGCA', 'GCATG']
k = 4

print("\nFlagged indexes:", flag_matching_kmer(seq_list, k))

Sequence:
0 {'ATGC', 'TGCA'}
1 {'CATG', 'GCAT'}

Reverse complement
0 {'GCAT', 'TGCA'}
1 {'ATGC', 'CATG'}

Flagged indexes: [0, 1]
